<a href="https://colab.research.google.com/github/JaeHeee/Pytorch_Tutorial/blob/main/code/PYTORCH_DEFINING_NEW_AUTOGRAD_FUNCTIONS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PYTORCH: DEFINING NEW AUTOGRAD FUNCTIONS

custom autograd Function을 구현하여 ReLU 기능을 수행하도록

In [ ]:
import torch

# torch.autograd.Function 상속받아서 custom autograd Functions 구현
# Tensor 연산을 하는forward and backward pass 구현
class MyReLU(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        # ctx는 context object로 backward 연산을 위한 정보 저장에 사용
        # ctx.save_for_backward method를 사용하여 backward pass 에서 사용할 어떠한 객체도 저장(cache) 가능
        ctx.save_for_backward(input)
        return input.clamp(min = 0)
    
    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input


dtype = torch.float
device = torch.device("cpu")

N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # custom function을 적용하기 위해 Function.apply 사용
    relu = MyReLU.apply
    y_pred = relu(x.mm(w1)).mm(w2)

    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    loss.backward()

    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
    
        w1.grad.zero_()
        w2.grad.zero_()  

99 579.15966796875
199 1.6684973239898682
299 0.0074860998429358006
399 0.00016995223995763808
499 3.117273445241153e-05
